In [7]:
#import the required libraries
import csv
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from IPython.display import display, HTML
import psycopg2

#pd.set_option("display.precision", 2)

# Graphics in retina format are more sharp and legible
%config InlineBackend.figure_format = 'retina'

In [8]:
# Read the csv file
df = pd.read_csv('c:/Users/ricki/Documents/Freelance/Projects/pgr.csv', delimiter=',')
df.head()

,asset,value,receiver,status,year
0,236,0.0,Ministério das Finanças,Propriedade do Estado,2024
1,235,0.0,Ministério das Finanças,Propriedade do Estado,2024
2,234,0.0,Ministério das Finanças,Propriedade do Estado,2024
3,233,0.0,Ministério das Finanças,Propriedade do Estado,2024
4,232,11110.0,BNA,à guarda do BNA,2024


In [9]:
df.tail()

,asset,value,receiver,status,year
837,6,0.0,CA da respectiva Sociedade,"Ministério das Telecomunicações, Tecnologias d...",2019
838,5,0.0,CA da respectiva Sociedade,Alteração do fiel depositário para o Ministéri...,2019
839,4,350000000.0,CA da respectiva Sociedade,Foi alterado o fiel depositário para o IGAPE e...,2019
840,3,630000000.0,CA da respectiva Sociedade,à guarda do fiel depositário,2019
841,2,420000000.0,CA da respectiva Sociedade,à guarda do fiel depositário,2019


In [10]:
desc = df.describe()
desc_formatted = desc.applymap(lambda x: f"{x:,.0f}")
print(desc_formatted)


      asset               value   year
count   842                 842    842
mean    204      59,115,587,458  2,021
std     156   1,002,805,026,113      1
min       1                   0  2,019
25%      79             322,000  2,020
50%     156           3,844,555  2,021
75%     310         900,907,500  2,021
max     559  22,942,642,018,900  2,024


In [11]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 842 entries, 0 to 841
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   asset     842 non-null    int64  
 1   value     842 non-null    float64
 2   receiver  842 non-null    object 
 3   status    842 non-null    object 
 4   year      842 non-null    int64  
dtypes: float64(1), int64(2), object(2)
memory usage: 33.0+ KB


In [12]:
df.isnull().sum()

asset       0
value       0
receiver    0
status      0
year        0
dtype: int64

In [13]:
df.describe(include=["object"])

,receiver,status
count,842,842
unique,38,29
top,Cofre Geral de Justiça,à guarda do fiel depositário
freq,356,554


In [14]:
df["receiver"].value_counts()

Cofre Geral de Justiça                                        356
Ministério das Finanças                                       167
Instituição Financeira Bancária                               112
ASCOFA                                                         57
BNA                                                            30
China Sonangol International Limited                           21
Fundo de Fomento Habitacional                                  18
IGAPE                                                          12
CA da respectiva Sociedade                                     10
Ministério do Comércio e Indústria                              8
INSS                                                            6
Ministério das Telecomunicações e Tecnologia de Informação      6
Ministério da Saúde                                             4
Direcção Nacional do Património do Estado                       4
Sonangol, E.P.                                                  3
Banco de P

In [15]:
df["status"].value_counts()

à guarda do fiel depositário                                                                                                    554
Propriedade do Estado                                                                                                           107
Entregue ao Estado                                                                                                               75
Afectado a uma instituição pública                                                                                               42
à guarda do BNA                                                                                                                  23
Entregue ao Ministério das Finanças                                                                                               5
Sob gestão do Ministério das Finanças                                                                                             5
Alteração do fiel depositário para o IGAPE em 2021                          

In [16]:
def run_query(query):
    try:
        conn = psycopg2.connect(
        user="*****",
        password="*****",
        host="000.0.0.0",
        port="****",
        database="*******"
        )
        cur = conn.cursor()
        print("Connected to PostgreSQL")

        #execute query
        cur.execute(query)

        rows = cur.fetchall()
        qdf = pd.DataFrame(rows, columns=[desc[0] for desc in cur.description])

        cur.close()
        conn.close()
        print("PostgreSQL connection is closed")
        
        return qdf

    except (Exception, psycopg2.Error) as error:
        print("Error while connecting to PostgreSQL", error)

#finally:
        #if conn:
        #    cur.close()
        #    conn.close()
        #    print("PostgreSQL connection is closed")

In [17]:
query = "SELECT * FROM finances"
result_df = run_query(query)
#print(result_df.head(5).to_string(index=True, justify='center'))
display(HTML(result_df.head().to_html()))

Connected to PostgreSQL
PostgreSQL connection is closed


,id,asset,value,receiver,status,year
0,1441,236,0.0,Ministerio das Financas,Propriedade do Estado,2024
1,1442,235,0.0,Ministerio das Financas,Propriedade do Estado,2024
2,1443,234,0.0,Ministerio das Financas,Propriedade do Estado,2024
3,1444,233,0.0,Ministerio das Financas,Propriedade do Estado,2024
4,1445,232,11110.0,BNA,a guarda do BNA,2024


In [18]:
query = "SELECT SUM(value) AS activos_apreendidos FROM finances WHERE id BETWEEN 1668 AND 2137"
result_df = run_query(query)
print(result_df)

Connected to PostgreSQL
PostgreSQL connection is closed
  activos_apreendidos
0    23294757092815.0


23,294,757,092,815.0

In [19]:
query = "SELECT SUM(value) AS activos_recuperados FROM finances WHERE id BETWEEN 1441 AND 1667"
result_df = run_query(query)

print(result_df)

Connected to PostgreSQL
PostgreSQL connection is closed
  activos_recuperados
0     2836622091777.0


2,836,622,091,777.0

In [20]:
query = "SELECT SUM(value) AS activos_arrestados FROM finances WHERE id BETWEEN 2138 AND 2282"
result_df = run_query(query)

print(result_df)

Connected to PostgreSQL
PostgreSQL connection is closed
  activos_arrestados
0   23643945454919.0


23,643,945,454,919.0